In [1]:
import pandas as pd
import jieba
from torchtext.vocab import FastText
import torch
import numpy as np

In [2]:
#是否使用gpu
use_cuda=False

In [3]:
def get_text(x):
    record=x["result"]
    t=''.join([str(i["type"]) for i in record])
    detail=''.join([str(i["detail"]) for i in record])
    other=''.join([str(i["other"]) for i in record])
    return "".join([t,detail,other])

In [4]:
class fasttext(torch.nn.Module):
    def __init__(self):
        super(fasttext,self).__init__()
        self.net=torch.nn.Sequential(
        torch.nn.Linear(300,256),
        torch.nn.ReLU(),
        torch.nn.Linear(256,32),
        torch.nn.ReLU(),
        torch.nn.Linear(32,2),
        torch.nn.Softmax(dim=1))
    
    def forward(self,x):
        out=self.net(x)
        return out

In [5]:
def bigram(x):
    #需要改一下torchtext的源码
    #找见所安装包下的vocab.py 410行
    #url_base替换成 'https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.{}.vec'
    vectors=FastText(language="zh")
    xx=[]
    for i in range(len(x)-1):
        xx+=[x[i]+x[i+1]]
    if use_cuda:
        return torch.mean(torch.stack(list(map(lambda y:vectors[y],xx))),dim=0)
    else:
        return torch.mean(torch.stack(list(map(lambda y:vectors[y],xx))),dim=0)

In [6]:
#vscode csv to json插件转换的
data_n=pd.read_json('./negative_500.json',orient='records')
data_p=pd.read_json('./positive_500.json',orient='records')

In [7]:
data_n["data"]=data_n.iloc[:,1].map(get_text)
data_p["data"]=data_p.iloc[:,1].map(get_text)

In [8]:
data=pd.concat([data_n,data_p])
data["embed"]=data["data"].map(bigram)
data=data.sample(frac=1)

In [25]:
#保存一下避免之后重跑上面的代码 很耗时……
data.to_excel(r"processed_data.xlsx",index=0)
data=pd.read_excel(r"processed_data.xlsx")

IllegalCharacterError: 

In [12]:
if use_cuda:
    train_x=torch.stack(list(data["embed"]),dim=0).to("cuda")
    train_y=torch.tensor(list(data["label"])).long().to("cuda")
else:
    train_x=torch.stack(list(data["embed"]),dim=0)
    train_y=torch.tensor(list(data["label"])).long()    
#ones=torch.sparse.torch.eye(2)
#train_y=ones.index_select(0,train_y).long()

In [19]:
if use_cuda:
    loss=torch.nn.CrossEntropyLoss(weight=torch.tensor([1.,5.]))
else:
    loss=torch.nn.CrossEntropyLoss(weight=torch.tensor([1.,5.]))
#loss=torch.nn.MSELoss
epoch_n=1000
learning_rate=1e-3


In [20]:
if use_cuda:
    model=fasttext().to("cuda")
else:
    model=fasttext()
optimzer = torch.optim.Adam(model.parameters(),lr = learning_rate)

In [21]:
ttrain_x=train_x[:800]
ttrain_y=train_y[:800]
pre_x=train_x[800:]
pre_y=train_y[800:]

In [22]:
def test(y,pred):
    l=loss(pred,y)
    print("Loss:{:.4f}".format(l.data),end='')
    ly=pred[:,0]<pred[:,1]
    TP=int(((y==1)&(ly==1)).sum())
    TN=int(((y==0)&(ly==0)).sum())
    FN=int(((y==1)&(ly==0)).sum())
    FP=int(((y==0)&(ly==1)).sum())
    if TP+FN!=0:
        r=TP/(TP+FN)
    else:
        r=-1
    if TP+FP!=0:
        p=TP/(TP+FP)
    else:
        p=-1
    print("p:{:.4f}".format(p),end='')
    print("r:{:.4f}".format(r),end='')
    if r>=0 and p >=0:
        print("acc:{:.4f}".format((TP+TN)/(TP+FP+FN+TN)),end='')
    if r!=0 or p!=0:
        print("F1:{:.4f}".format(2*r*p/(r+p)),end='')
    print("")

In [23]:
def te(m,mode=0):
    if mode==0:
        pred=m(pre_x)
        test(pre_y,pred)
    else:
        pred=m(ttrain_x)
        test(ttrain_y,pred)

In [24]:
for epoch in range(epoch_n):
    y_pred=model(ttrain_x)
    #print(y_pred)
    l=loss(y_pred,ttrain_y)
    if epoch%100==0:
        print("Epoch:{},Loss:{:.4f}".format(epoch,l.data))
#     if epoch%100==0:
        print("training set:",end='')
        te(model,1)
        print("test set:",end='')
        te(model,0)
        
    optimzer.zero_grad()
    l.backward()
    #for param in model.parameters():
    #    param.data-=param.grad.data*lr
    optimzer.step()

Epoch:0,Loss:0.7078
training set:Loss:0.7078p:-1.0000r:0.0000F1:0.0000
test set:Loss:0.7066p:-1.0000r:0.0000F1:0.0000
Epoch:100,Loss:0.4745
training set:Loss:0.4745p:0.5100r:1.0000acc:0.5100F1:0.6755
test set:Loss:0.5037p:0.4600r:1.0000acc:0.4600F1:0.6301
Epoch:200,Loss:0.4737
training set:Loss:0.4737p:0.5119r:1.0000acc:0.5138F1:0.6772
test set:Loss:0.5034p:0.4600r:1.0000acc:0.4600F1:0.6301
Epoch:300,Loss:0.4731
training set:Loss:0.4731p:0.5132r:1.0000acc:0.5162F1:0.6783
test set:Loss:0.5032p:0.4600r:1.0000acc:0.4600F1:0.6301
Epoch:400,Loss:0.4519
training set:Loss:0.4519p:0.5605r:0.9877acc:0.5988F1:0.7152
test set:Loss:0.5050p:0.4972r:0.9565acc:0.5350F1:0.6543
Epoch:500,Loss:0.4411
training set:Loss:0.4411p:0.5870r:0.9926acc:0.6400F1:0.7377
test set:Loss:0.5126p:0.5091r:0.9130acc:0.5550F1:0.6537
Epoch:600,Loss:0.4291
training set:Loss:0.4291p:0.6560r:0.9583acc:0.7225F1:0.7789
test set:Loss:0.5092p:0.5870r:0.8804acc:0.6600F1:0.7043
Epoch:700,Loss:0.4153
training set:Loss:0.4153p:0.6473